In [1]:
import paho.mqtt.client as mqtt
import time
import paho.mqtt.publish as publish
from collections import deque
class Controllee:
    HOST = "mqtt"

    PORT = 1883
    
    def __init__(self,mqtt_config=("mqtt",1883,("tom","xiao"))):
        self.HOST,self.PORT,self.client_pw=mqtt_config
        self.client=self.client_start()
        self.client.loop_start()
    
    def client_start(self):
        client = mqtt.Client(str(time.time()))
        client.username_pw_set(*self.client_pw)
        client.on_connect = self.on_connect
        client.on_message = self.on_message
        client.connect(self.HOST, self.PORT, 60)
        return client

    def on_connect(self,client, userdata, flags, rc):
        print("Connected with result code "+str(rc))
        client.subscribe("control")
        
    def __call__(self,*args):
        self.publish(*args)
        
    def publish(self,thing):
        self.client.publish("control_back",thing)
        
    result=deque((),10)
    #TODO: possible too_much
    #to not flood result_keeper with too much out-dated info
    def on_message(self,client, userdata, msg):
#         print(msg.payload)
        self.result.append(msg.payload)
#         C=msg.payload.decode("utf-8")
#         result.append(C)
        
    def pop_result(self):
        try:return self.result.popleft()
        except:pass
        

In [2]:
C=Controllee()

Connected with result code 0


In [3]:
class State:pass
s=State()

In [4]:
import asyncio

In [5]:
async def result_keeper(state,get_from,sleep_interval=0.001,keep_len=10,to_value_name="result"):
    assert callable(get_from)
    from collections import deque
    
    result=deque((),keep_len)
    setattr(state,to_value_name,result)
    
    while 1:
        await asyncio.sleep(sleep_interval)
        got=get_from()
        if got is None:continue
        result.append(got)
rk=asyncio.create_task(result_keeper(s,C.pop_result))

In [6]:
async def result_runner(state,C,keep_len=10,
                     from_value_name="result",
                     sync_time=0.01):
    while 1:
        await asyncio.sleep(sync_time)
        if hasattr(state,from_value_name):break
    de=getattr(state,from_value_name)
    
    from collections import deque
    ret=deque((),keep_len)
#     setattr(state,to_value_name,result)
    
    while 1:
        await asyncio.sleep(sync_time)
        try:x=de.popleft()
        except:continue
        try:exec(x)
        except Exception as e:
            C.publish("error")
            C.publish(str(e))
        try:g=ret.popleft()
        except:continue
        print(g)
        C.publish(g)

rr=asyncio.create_task(result_runner(s,C))

In [7]:
s.result

AttributeError: 'State' object has no attribute 'result'

In [ ]:
rk.result()

In [ ]:
rr.result()

In [ ]:
a,b,c,d

In [ ]:
a

In [ ]:
a

b'ret.append(1)'
1
b'ret=2'
b'ret.append(1)'
1
b'ret.append(1)'
1
b'ret=2'
b'ret.append(1)'
1
b'hihi'
